# ADD mem

In [3]:
import os
from mem0 import Memory
from dotenv import load_dotenv
load_dotenv()

SHARED_OPEN_AI_CONFIG = {  
    "llm": {  
        "provider": "openai",  
        "config": {  
            "model": os.getenv("MODEL", "gpt-4o"),  
            "api_key": os.getenv("OPENAI_API_KEY")  
        }  
    },  
    "embedder": {  
        "provider": "openai",  
        "config": {  
            "model": "text-embedding-3-small"  
        }  
    },  
    "vector_store": {  
        "provider": "faiss",  
        "config": {  
            "path": "./faiss_graph_db",  
            "collection_name": "mem0_evaluation"  
        }  
    }
}
m = Memory.from_config(SHARED_OPEN_AI_CONFIG)
messages = [
    {"role": "user", "content": "I'm planning to watch a movie at 11/11/2024. Any recommendations?"},
    {"role": "assistant", "content": "How about a thriller movies? They can be quite engaging."},
    {"role": "user", "content": "I'm not a big fan of thriller movies but I love sci-fi movies."},
    {"role": "assistant", "content": "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future."}
]

# Store inferred memories (default behavior)
result = m.add(messages, user_id="alice", metadata={"category": "movie_recommendations"})
print("Inferred Memory Result:", result)
# Optionally store raw messages without inference
result = m.add(messages, user_id="Tom", metadata={"category": "movie_recommendations"}, infer=False)
print("Raw Memory Result:", result)

Inferred Memory Result: {'results': [{'id': '3e5ed206-ef8b-401c-bdd7-ae3ec7dbbd5c', 'memory': 'Planning to watch a movie on 11/11/2024', 'event': 'ADD'}, {'id': '9afa8e0e-e405-483a-97a7-2bb1afd73ea2', 'memory': 'Not a big fan of thriller movies', 'event': 'ADD'}, {'id': '28acd5b4-75ff-4f78-9c93-0da51f9a93c1', 'memory': 'Loves sci-fi movies', 'event': 'ADD'}]}
Raw Memory Result: {'results': [{'id': 'a911d471-4f25-4fc2-b520-f7dd07c6a710', 'memory': "I'm planning to watch a movie at 11/11/2024. Any recommendations?", 'event': 'ADD', 'actor_id': None, 'role': 'user'}, {'id': '501c6d98-1827-4f4e-8079-82fccda831ab', 'memory': 'How about a thriller movies? They can be quite engaging.', 'event': 'ADD', 'actor_id': None, 'role': 'assistant'}, {'id': 'daa2b6c8-8dc2-434f-bd1c-5cd9bb34f369', 'memory': "I'm not a big fan of thriller movies but I love sci-fi movies.", 'event': 'ADD', 'actor_id': None, 'role': 'user'}, {'id': '6c1e052d-f109-40a5-b215-089ebf9c92e6', 'memory': "Got it! I'll avoid thril

# update - delete: memory

In [9]:
messages = [
    {"role": "user", "content": "Actually, I'm planning to watch a football match at 11/11/2024, not movies."},
]
# Store inferred memories (default behavior)
result = m.add(messages, user_id="alice")
print("Inferred Memory Result:", result)

Inferred Memory Result: {'results': [{'id': 'eb9e3e4e-99be-49ed-9845-b90732127ab7', 'memory': 'Planning to watch a football match on 11/11/2024', 'event': 'ADD'}]}


In [6]:
messages = [
    {"role": "user", "content": "I love watching movies, especially sci-fi."},
]
# Store inferred memories (default behavior)
result = m.add(messages, user_id="alice", metadata={"category": "movie_recommendations"})
print("Inferred Memory Result:", result)

Inferred Memory Result: {'results': [{'id': '28acd5b4-75ff-4f78-9c93-0da51f9a93c1', 'memory': 'Loves watching movies, especially sci-fi', 'event': 'UPDATE', 'previous_memory': 'Loves sci-fi movies'}]}


In [7]:
messages = [
    {"role": "user", "content": "Actually, I do not love watching movies, especially sci-fi."}

]
# Store inferred memories (default behavior)
result = m.add(messages, user_id="alice", metadata={"category": "movie_recommendations"})
print("Inferred Memory Result:", result)

Inferred Memory Result: {'results': [{'id': '28acd5b4-75ff-4f78-9c93-0da51f9a93c1', 'memory': 'Loves watching movies, especially sci-fi', 'event': 'DELETE'}]}


# search

In [8]:

related_memories = m.search("Should I drink coffee or tea?", user_id="Tom")

related_memories

{'results': [{'id': 'a911d471-4f25-4fc2-b520-f7dd07c6a710',
   'memory': "I'm planning to watch a movie at 11/11/2024. Any recommendations?",
   'hash': 'ebdcf7812a397c3f0e4b1a3a9f6877dd',
   'metadata': {'category': 'movie_recommendations'},
   'score': 1.6732146739959717,
   'created_at': '2025-07-08T03:11:08.315244-07:00',
   'updated_at': None,
   'user_id': 'Tom',
   'role': 'user'},
  {'id': '501c6d98-1827-4f4e-8079-82fccda831ab',
   'memory': 'How about a thriller movies? They can be quite engaging.',
   'hash': '6b8582d760dab9091db299ac5a8cf759',
   'metadata': {'category': 'movie_recommendations'},
   'score': 1.7241477966308594,
   'created_at': '2025-07-08T03:11:09.433660-07:00',
   'updated_at': None,
   'user_id': 'Tom',
   'role': 'assistant'}]}

# Custom fact extraction promt

In [3]:
custom_fact_extraction_prompt = """
Please only extract entities containing customer support information, order details, and user information. 
Here are some few shot examples:

Input: Hi.
Output: {{"facts" : []}}

Input: The weather is nice today.
Output: {{"facts" : []}}

Input: My order #12345 hasn't arrived yet.
Output: {{"facts" : ["Order #12345 not received"]}}

Input: I'm John Doe, and I'd like to return the shoes I bought last week.
Output: {{"facts" : ["Customer name: John Doe", "Wants to return shoes", "Purchase made last week"]}}

Input: I ordered a red shirt, size medium, but received a blue one instead.
Output: {{"facts" : ["Ordered red shirt, size medium", "Received blue shirt instead"]}}

Return the facts and customer information in a json format as shown above.
"""

In [6]:
from mem0 import Memory
import os
from dotenv import load_dotenv
load_dotenv()
config = {
    "llm": {
        "provider": "gemini",
        "config": {
            "model": "gemini-2.0-flash-001",
            "temperature": 0.2,
            "max_tokens": 2000,
            "top_p": 1.0,
            "api_key": os.getenv("GOOGLE_API_KEY")
        }
    },
    "embedder": {
        "provider": "huggingface",
        "config": {
            "model": "multi-qa-MiniLM-L6-cos-v1"
        }
    },
    "vector_store": {  
        "provider": "faiss",  
        "config": {  
            "path": "./faiss_graph_db",  
            "collection_name": "mem0_evaluation"  ,
            "embedding_model_dims": 384
        }  
    },
    "custom_fact_extraction_prompt": custom_fact_extraction_prompt,
    "version": "v1.1"
}

m = Memory.from_config(config_dict=config)

In [7]:
m.add("Yesterday, I ordered a laptop, the order id is 12345", user_id="alice")

{'results': [{'id': 'e2ceacfa-c9dd-44ff-a4a9-ff0ca6fc1d4a',
   'memory': 'Ordered laptop yesterday',
   'event': 'ADD'},
  {'id': '74771112-4060-45ea-a432-121c31255fac',
   'memory': 'Order ID: 12345',
   'event': 'ADD'}]}

In [8]:
m.add("I like going to hikes", user_id="alice")

ERROR:root:Error in new_retrieved_facts: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)


{'results': []}